In [ ]:
#| default_exp basic

In [ ]:
#| export
from __future__ import annotations

# basic
> basic helpers

# Prologue


In [ ]:
#| export

import pprint
from typing import Iterable
from typing import Self
from typing import TypeVar

import fastcore.all as FC


In [ ]:
from types import NoneType

from fastcore.test import *


In [ ]:
from olio.test import *


----

# AD

In [ ]:
#| export

_VT = TypeVar('_VT')
# from `fastcore` + generics
class AD(dict[str, _VT]):
    "`dict` subclass that also provides access to keys as attrs"
    def __getattr__(self, k:str) -> _VT: return self[k] if k in self else FC.stop(AttributeError(k))  # type: ignore
    def __setattr__(self, k, v:_VT): (self.__setitem__, super().__setattr__)[k[0]=='_'](k,v)
    def __dir__(self) -> Iterable[str]: return super().__dir__() + list(self.keys())  # type: ignore
    def _repr_markdown_(self): return f'```json\n{pprint.pformat(self, indent=2)}\n```'
    def copy(self) -> Self: return type(self)(**self)


# is_listy
> Test whether `x` is list-like


In [ ]:
#| export

def is_listy(x):
    return isinstance(x, Iterable) and not isinstance(x, (bytes, str))

def is_listy_type(x):
    return issubclass(x, Iterable) and not issubclass(x, (bytes, str))


In [ ]:

for _ in (
    [1, 2, 3], (1, 2, 3), {1, 2, 3}, 
    {'a': 1}, range(3), (i for i in range(3)), FC.L(1, 2, 3),
    ):
    test_is(is_listy(_), True)
for _ in (
    'a', b'a', r'a', None,
    ):
    test_is_not(is_listy(_), True)

for _ in (list, tuple, set, dict, range, type(i for i in range(3)), FC.L):
    test_is(is_listy_type(_), True)
for _ in (str, bytes, type(r'a'), NoneType,):
    test_is_not(is_listy_type(_), True)

# flatten


In [ ]:
#| export

def flatten(o):
    "Concatenate all collections and items as a generator"
    for item in o:
        if not is_listy(item): yield item; continue
        try: yield from flatten(item)
        except TypeError: yield item


In [ ]:
for _, expected in (
    ([], []),  
    ([1, 2, 3], (1, 2, 3)), 
    ((1, (2, 3)), (1, 2, 3)), 
    ([('a', (2,)), ('c', (4,))], ('a', 2, 'c', 4)), 
):
    test_eq(flatten(_), expected)



# shorten
> truncate string


In [ ]:
#| export

def shorten(x, mode='l', limit=40, trunc='…', empty='') -> str:
    s = str(x)
    if len(s) > limit:
        # s = f'{s[:limit//2]} ... {s[:-limit//2]}'
        # s = s[:limit] + '..' * (len(s) > limit)
        # s = (
        #         f'...{s[sl1:sl2]}'
        #         if mode == 'l' else 
        #         f's[sl1:sl2]...'
        #     )
        # sl1, sl2, l, r = (-limit, None, trunc, empty) if mode == 'l' else (None, limit, empty, trunc)
        # s = f'{l}{s[sl1:sl2]}{r}'
        l, m, r = (
            (empty, trunc, s[-limit:]) if mode == 'l' else 
            (s[:limit], trunc, empty) if mode == 'r' else 
            (s[:(limit//2)-1], f" {trunc} ", s[-(limit//2-1):])
        )
        s = f'{l}{m}{r}'
    return s


In [ ]:
test_eq(f"{shorten(0)}", '0')
test_eq(f"{shorten(234)}", '234')
test_eq(f"{shorten('asdfgh')}", 'asdfgh')
test_eq(f"{shorten('It was the best of times', limit=12)}", '…est of times')
test_eq(f"{shorten('ad2663b4-5ff5-40e3-a6ed-cc35f5627f8d', limit=17)}", '…a6ed-cc35f5627f8d')
test_eq(f"{shorten('ad2663b4-5ff5-40e3-a6ed-cc35f5627f8d', 'r', limit=17)}", 'ad2663b4-5ff5-40e…')
test_eq(f"{shorten('ad2663b4-5ff5-40e3-a6ed-cc35f5627f8d', 'c', limit=17)}", 'ad2663b … 5627f8d')


# Colophon
----

In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '00_basic.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
